In [14]:
import cv2
import os
import pandas as pd

In [15]:
def describe_images(folder):
    desc=[]
    for i in os.listdir(folder):
        image =folder+'/'+i
        img =  cv2.imread(image)
        height, width, depth = img.shape
        desc.append({'file':image,'w':width,'h':height,'aspect_ratio':height/width})
        
    desc=pd.DataFrame(desc)
    
    return desc
    #w=max(ws) #max width full stub images
    #h=max(hs) #max width full stub images     

In [16]:
desc=describe_images('images/thumbs')

In [17]:
desc

,file,w,h,aspect_ratio
0,images/thumbs/12.jpg,360,225,0.625
1,images/thumbs/08.jpg,360,225,0.625
2,images/thumbs/02.jpg,360,225,0.625
3,images/thumbs/01.jpg,360,225,0.625
4,images/thumbs/06.jpg,360,225,0.625
5,images/thumbs/09.jpg,360,225,0.625
6,images/thumbs/04.jpg,360,225,0.625
7,images/thumbs/03.jpg,360,225,0.625
8,images/thumbs/07.jpg,360,225,0.625
9,images/thumbs/11.jpg,360,225,0.625


In [18]:
thumb_desc=describe_images('images/thumbs')

In [19]:
def resize(img,df):
    height, width, depth = img.shape
    aspect_ratio = height/width #actual aspect ratio
    print('original size ---> height: '+str(height)+' width: '+str(width))
    df=df.iloc[(df['aspect_ratio']-height).abs().argsort()[:1]] #closest aspect ratio
    w=df.w.iloc[0]
    h=df.h.iloc[0]
    if height>width:
        dimension = (int(h/aspect_ratio),h)
    else:
        dimension = (w,int(w*aspect_ratio))
    
    print('new size ---> height: '+str(dimension[0])+' width: '+str(dimension[1]))
    return cv2.resize(img, dimension)
    #return dimension

In [20]:
def thumbnail(img,desc):
    df=desc.copy()
    height, width, depth = img.shape
    print('original image dim---> height:'+str(height)+' width: '+str(width))
    aspect_ratio = height/width #actual aspect ratio
    h=df.loc[0,'h'] #fixed height
    if width>height:
        w=df.loc[df['aspect_ratio'].idxmin(),'w'] #select row with higher aspect_radio, set new width to this value        
        if int(w*aspect_ratio)>h:
            dimension=(w,int(w*aspect_ratio))
        else:
            dimension=(int(h/aspect_ratio),h)
    else:
        w=df.loc[df['aspect_ratio'].idxmax(),'w'] #select row with lower aspect_radio, set new width to this value
        if int(w*aspect_ratio)>h:
            dimension=(w,int(w*aspect_ratio))
        else:
            dimension=(int(h/aspect_ratio),h)
    print(dimension)
    img=cv2.resize(img, dimension)
    height, width, depth = img.shape
    print('resized image dim---> height:'+str(height)+' width: '+str(width))    
    

    img=img[
            int(height/2)-int(h/2):int(height/2)+int(h/2),
    int(width/2)-int(w/2):int(width/2)+int(w/2)]
    height, width, depth = img.shape
    print('final thumbnail dim---> height:'+str(height)+' width: '+str(width))
    return img

In [21]:
#thumbnail(img,thumb_desc)

In [22]:
def show_image(image):
    if type(image)==str:
        img =  cv2.imread(image)
    else:
        img=image
    cv2.imshow("Original", img)
    cv2.waitKey(0)
    cv2.destroyWindow('Original')

In [23]:
path='images/fulls'

In [24]:
#show_image('images/Francisco Humeres - Twitter Maps_files/madrid___235K.png')

In [40]:
def caption(img,caption,x,y,cap_color='white'):
    height, width, depth = img.shape
    
    if cap_color=='white':
        BLACK = (255,255,255)
    else:
        BLACK = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_size = 0.5
    font_color = BLACK
    font_thickness = 1
    text = caption
    x,y = int(width*x),int(height*y)
    img = cv2.putText(img, text, (x,y), font, font_size, font_color, font_thickness, cv2.LINE_AA)
    
    return img

In [46]:
def batch_resize(source='',target='',thumb_target='',thumb_model='',model='',cap=''):
    desc=describe_images(model)
    thumb_desc=describe_images(thumb_model)
    for i in os.listdir(source):
        image =source+'/'+i
        print('processing image:'+image)
        img =  cv2.imread(image)
        
        print('generating thumbnail')
        thumb=thumbnail(img,thumb_desc)
        
        print('resizing image')
        img=resize(img,desc)
        lista=[1,2,3,4]
        y=[x for x in lista if(str(x) in i)]
        print(y)

        if len(y)>0:
            print('adding caption')
            if y[0]==1 or y[0]==3 :
                cap_color='white'
            else:
                cap_color='black'
            img=caption(img,cap,0.07,0.95,cap_color=cap_color)
        print('saving file '+target+'/'+i)
        cv2.imwrite(target+'/'+i,img)
        print('saving file '+thumb_target+'/'+i)
        cv2.imwrite(thumb_target+'/'+i,thumb)

In [47]:
batch_resize(thumb_target='images/tmp/thumbs',thumb_model='images/thumbs',source='images/Francisco Humeres - Eigencities_files',target='images/tmp/fulls',model='images/fulls',cap='(c) 2015-16 Francisco J. Humeres M.')

processing image:images/Francisco Humeres - Eigencities_files/intro.JPG
generating thumbnail
original image dim---> height:1513 width: 1920
(360, 283)
resized image dim---> height:283 width: 360
final thumbnail dim---> height:224 width: 360
resizing image
original size ---> height: 1513 width: 1920
new size ---> height: 1200 width: 945
[]
saving file images/tmp/fulls/intro.JPG
saving file images/tmp/thumbs/intro.JPG
processing image:images/Francisco Humeres - Eigencities_files/1-imagen_objetivo.png
generating thumbnail
original image dim---> height:1443 width: 1920
(360, 270)
resized image dim---> height:270 width: 360
final thumbnail dim---> height:224 width: 360
resizing image
original size ---> height: 1443 width: 1920
new size ---> height: 1200 width: 901
[1]
adding caption
saving file images/tmp/fulls/1-imagen_objetivo.png
saving file images/tmp/thumbs/1-imagen_objetivo.png
processing image:images/Francisco Humeres - Eigencities_files/3-methods.jpg
generating thumbnail
original im